### Concurrent Fibonacci with Message Passing (Go)

> "A certain man put a pair of rabbits in a place surrounded on all sides by a wall. How many pairs of rabbits can be produced from that pair in a year if it is supposed that every month each pair begets a new pair which from the second month on becomes productive?" – Leonardo of Pisa, later known as Fibonacci

The answer to that is the Fibonacci sequence. The numbers of that sequence are recursively defined by:

    fib(0)  =  0
    fib(1)  =  1
    fib(n)  =  fib(n – 1) + fib(n – 2)    for n > 1

The task is to compute the first `N = 20` Fibonacci numbers by `N` processes (goroutines), each computing `fib(i)` for `0 ≤ i < N` and communicating over an array `c` of `N` channels `c` and a channel `r`:
- Process `0` outputs the pair `0, 0` on `r` and `0` on `c[0]`.
- Process `1` outputs the pair `1, 1` on `r` and `1` on `c[1]`.
- Process `i`, for `1 < i < N`, receives numbers from `c[i – 1]` and `c[i – 2]`, computes their sum, outputs `i` and the sum on `r`, and the sum on `c[i]`.

The main program collects and compares all computed pairs to the expected pairs. In

    fib(2) = fib(1) + fib(0)
    fib(3) = fib(2) + fib(1)
    fib(4) = fib(3) + fib(2)
    ...
    fib(N - 2) = fib(N - 3) + fib(N - 4)
    fib(N - 1) = fib(N - 2) + fib(N - 3)

the terms `fib(0)` and `fib(N – 2)` occur only once on the right-hand sides, `fib(N – 1)` does not occur at all on the right-hand sides, and each `fib(i)` for `0 < i < N – 2` occurs twice. In the implementation, this means that only one “downstream process” receives from channels `0` and `N – 2`, no process receives from channel `N - 1`, and two processes receive from each of the other channels. That is, processes `fib(0)` and `fib(N – 2)` should send their computed Fibonacci number only once over their channel, `fib(N – 1)` should not send it at all, and all other processes should send it twice.

The template below compares the first 20 computed Fibonacci numbers with the expected ones by storing each in a map and comparing the two maps. The order of insertion in the map of computed numbers does not matter. Comparing two maps with `==` in Go compares only their addresses. The test below uses a function of the `reflect` library for *deep comparison*. [8 points]

In [17]:
%%writefile fibonacci.go
package main
import "reflect"

const N = 20

type IntPair = struct {i int; f int}

var r chan IntPair
var c[N] chan int

func fib(i int) {
	var f int
	if i == 0 {
		f = 0
		c[0] <- 0
	} else if i == 1 {
		f = 1
		c[1] <- 1
		c[1] <- 1
	} else {
		f1 := <-c[i-1]
		f2 := <-c[i-2]
		f = f1 + f2
		if i == N-2 {
			c[i] <- f
		} else if i != N-1 {
			c[i] <- f
			c[i] <- f
		}
	}

	r <- IntPair{i, f}
}

func main() {
    m := make(map[int]int)
    r = make(chan IntPair)
    for i := 0; i < N; i++ {c[i] = make(chan int)}
    for i := 0; i < N; i++ {go fib(i)}
    for i := 0; i < N; i++ {p := <- r; m[p.i] = p.f}
    e := map[int]int{0:0, 1:1, 2:1, 3:2, 4:3, 5:5, 6:8, 7:13, 8:21, 9:34, 10:55, 11:89,
        12:144, 13:233, 14:377, 15:610, 16:987, 17:1597, 18:2584, 19:4181}
    println("computed and expected Fibonacci numbers are the same: ", reflect.DeepEqual(m, e))
}

Overwriting fibonacci.go


In [18]:
!go run fibonacci.go

computed and expected Fibonacci numbers are the same:  true
